In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point


In [3]:
zipcodes = gpd.read_file('../../../../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head()

EPSG:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [4]:
zipcodes_cart = zipcodes.to_crs({'init':'epsg:3857'})
print(zipcodes_cart.crs)

+init=epsg:3857 +type=crs


C:\Users\rose\anaconda3\envs\scooters\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


this espg is cartesian which a

In [5]:
print(zipcodes_cart.crs)

+init=epsg:3857 +type=crs


In [6]:
zipcodes_cart['area'] = zipcodes_cart['geometry'].area/10**6

In [7]:
zipcodes_cart.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry,area
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-9649980.414 4344495.673, -964...",85.404945
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-9655241.695 4333382.848, -965...",27.015486
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-9660209.482 4319108.081, -966...",28.620981
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-9663931.872 4309319.397, -966...",24.917208
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-9687241.027 4302272.313, -968...",6.681648


In [9]:
zipcodes_cart['area'] = zipcodes_cart['area'] / 2.58998811

In [10]:
zipcodes_cart.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry,area
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-9649980.414 4344495.673, -964...",32.975034
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-9655241.695 4333382.848, -965...",10.430738
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-9660209.482 4319108.081, -966...",11.050622
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-9663931.872 4309319.397, -966...",9.620588
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-9687241.027 4302272.313, -968...",2.579799


In [11]:
selection_area = zipcodes[0:5]

In [14]:
for index, row in selection_area.iterrows():
    poly_area = row['geometry'].area
    print("Polygon area at index {0} is {1: .3f}".format(index, poly_area))

Polygon area at index 0 is  0.006
Polygon area at index 1 is  0.002
Polygon area at index 2 is  0.002
Polygon area at index 3 is  0.002
Polygon area at index 4 is  0.000


 Line below is reprojecting shape in Cylindrical equal-area format with {'proj':'cea'} that preserve area measure.

In [22]:
zipcodes_test = zipcodes['geometry'].to_crs({'proj': 'cea'})
type(zipcodes_test)

KeyError: 'geometry'

In [17]:
(zipcodes_test.area / 10**6) /  2.58998811

0     21.399563
1      6.775731
2      7.198472
3      6.277082
4      1.684972
5      0.068172
6      0.002028
7     45.429851
8     39.277807
9      2.651732
10     2.680024
11     0.010927
12    35.921720
13     7.947122
14     0.877504
15     0.119712
16     0.000208
17    18.567161
18    24.868482
19     0.250654
20    31.144623
21     0.019852
22    14.292525
23     1.154236
24     0.236007
25     0.005597
26     0.014924
27    38.453870
28     0.024244
29     0.002809
30     0.085817
31     0.839145
32    14.384320
33     0.382525
34     3.452994
35     4.328048
36     0.400029
37    24.180670
38     9.326879
39     0.101947
40     4.584933
41     0.039677
42    19.972658
43     0.015568
44    15.238042
45     1.717514
46    35.291930
47     9.529224
48     0.002814
49    21.603456
50     1.065417
51    41.274823
52     0.001397
53     8.458997
54     1.514681
55     0.084789
dtype: float64

In [18]:
zipcodes = zipcodes.geometry.to_crs({'proj':'cea'})

In [19]:
zipcodes['area_m2'] = (zipcodes.area / 10**6) /  2.58998811

geopandas.geoseries.GeoSeries